# MSSQL Integration Demo

### Imports & Setup

In [ ]:
%load_ext autoreload
%autoreload 2
from datetime import datetime
from zoneinfo import ZoneInfo
from helpers.irp_integration import IRPClient
from helpers.sqlserver import execute_query_from_file, execute_script_file

irp_client = IRPClient()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### Setup

In [ ]:
peril = "USEQ"
edm_name = f"{peril}_Automated"
portfolio_name = f"{peril}_Automated"

edms = irp_client.edm.search_edms(filter=f"exposureName=\"{edm_name}\"")
if (len(edms) != 1):
    raise Exception(f"Found {len(edms)} EDMs with name {edm_name}; expected 1")
edm = edms[0]
exposure_id = edm['exposureId']
edm_full_name = edm['databaseName']
print(f'EDM Full Name: {edm_full_name}')

portfolios = irp_client.portfolio.search_portfolios(exposure_id, f"portfolioName=\"{portfolio_name}\"")
if (len(portfolios) != 1):
    raise Exception(f"Found {len(edms)} Portfolios with name {portfolio_name}; expected 1")
portfolio_id = portfolios[0]['portfolioId']
print(f'Portfolio ID: {portfolio_id}')

EDM Full Name: USEQ_Automated_rGLG
Portfolio ID: 1


### Portfolio Mapping

In [ ]:
connection_name = "DATABRIDGE"
sql_script_name = f"2b_Query_To_Create_Sub_Portfolios_{peril}_RMS_BackEnd.sql"
params = {
    "EDM_FULL_NAME": edm_full_name,
    "PORTFOLIO_ID": portfolio_id,
    "DATETIME_VALUE": datetime.now(ZoneInfo("America/New_York")).strftime("%Y-%m-%d %H:%M:%S.%f")[:-3]
}

row_count = execute_script_file (
    f'portfolio_mapping/{sql_script_name}',
    params = params,
    connection = connection_name
)

print(f'Portfolio Mapping executed for Portfolio: {portfolio_name}')

Portfolio mapping executed; rows affect: 0


### Execute Simple SQL Scripts

In [ ]:
connection_name = "DATABRIDGE"
sql_script_name = "list_edm_tables.sql"

dataframes = execute_query_from_file(
    f'examples/{sql_script_name}',
    params={'EDM_FULL_NAME': edm_full_name},
    connection=connection_name
)

for df in dataframes:
    print(f"Query returned {len(df)} rows")
    print("\nResults:")
    print(df)

In [ ]:
connection_name = "DATABRIDGE"
sql_script_name = "test_multiple_select.sql"

dataframes = execute_query_from_file(
    f'examples/{sql_script_name}',
    params={'EDM_FULL_NAME': edm_full_name},
    connection=connection_name
)

for df in dataframes:
    print(f"Query returned {len(df)} rows")
    print("\nResults:")
    print(df.head())